In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import json

In [2]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-7B-Chat", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-7B-Chat", device_map="auto", trust_remote_code=True).eval()
model.generation_config = GenerationConfig.from_pretrained("Qwen/Qwen-7B-Chat", trust_remote_code=True)

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [3]:
# write a function to translate paragraphs
# let's write a function that can add English translations to our file for a given chapter in a book

def translate_chapter(model, tokenizer, book_path, chapter_number, prompt=None):
    if not prompt:
        prompt = "答案不能包含任何汉字。Do not include any Chinese characters in your output. Only write responses in English. Directly translate the following from Chinese direclty into English:"
    # open the file
    with open(f"../data/books/{book_path}/chapters/{chapter_number}.json", "r") as f:
        chapter = json.loads(f.read())
        translated_paragraphs = []
        for paragraph in chapter["paragraphs"]:
            full_prompt = f"{prompt}: {paragraph}"
            response, _ = model.chat(tokenizer, full_prompt, history=None)
            print(paragraph)
            print()
            print(response)
            print("======")
            print()
            translated_paragraphs.append(response)

        chapter["translated_paragraphs"] = translated_paragraphs

    with open(f"../data/books/{book_path}/chapters/{chapter_number}.json", "w") as f:
        json.dump(chapter, f, ensure_ascii=False)

    print(f"translated {len(translated_paragraphs)}.")

In [ ]:
for i in range(32,37): 
    print(f"===Translating chapter {i}===")
    translate_chapter(model, tokenizer, "three_body", i)
    print(f"===Finished translating {i}")